<a href="https://colab.research.google.com/github/siddhi1991/GIS-programming/blob/main/Interactive_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install rasterio
import rasterio


In [20]:
!pip install localtileserver

In [21]:
!pip install fiona

In [22]:
!pip install leafmap

In [23]:
import leafmap
import pandas as pd


In [24]:
m = leafmap.Map(center=[40, -95], zoom=5)

In [25]:
m.add_basemap("Esri.WorldImagery")

In [26]:
url = "https://www.mrlc.gov/geoserver/mrlc_display/NLCD_2021_Land_Cover_L48/wms?"
m.add_wms_layer(
    url=url,
    layers="NLCD_2021_Land_Cover_L48",
    name="NLCD 2021",
    attribution="MRLC",
    format="image/png",
    shown=True,
)


In [27]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

# Download a test GeoTIFF
import urllib.request
url = "https://github.com/giswqs/data/raw/main/raster/srtm90.tif"
input_tif = "srtm90.tif"
urllib.request.urlretrieve(url, input_tif)

# Reproject to Lambert Conformal Conic (EPSG:3978)
dst_tif = "srtm90_lambert.tif"
dst_crs = "EPSG:3978"

with rasterio.open(input_tif) as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    with rasterio.open(dst_tif, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)

print("Reprojection complete. Saved as:", dst_tif)


Reprojection complete. Saved as: srtm90_lambert.tif


In [28]:
import geopandas as gpd

# Load shapefile
shapefile_url = "https://github.com/giswqs/leafmap/raw/master/examples/data/us_states.shp"
gdf = gpd.read_file(shapefile_url)

# Reproject using manual LCC definition
lcc_proj = (
    "+proj=lcc +lat_1=33 +lat_2=45 +lat_0=39 +lon_0=-96 "
    "+x_0=0 +y_0=0 +datum=NAD83 +units=m +no_defs"
)
gdf_lambert = gdf.to_crs(lcc_proj)

# Save and verify
gdf_lambert.to_file("us_states_lambert.shp")
print(gdf_lambert.crs)


+proj=lcc +lat_1=33 +lat_2=45 +lat_0=39 +lon_0=-96 +x_0=0 +y_0=0 +datum=NAD83 +units=m +no_defs +type=crs


In [29]:
markers = [
    {"state": "Iowa", "latitude": 42.0, "longitude": -93.0, "area": "Maximum Cultivated Crops"},
    {"state": "Nebraska", "latitude": 41.125, "longitude": -98.268, "area": "Maximum Cultivated Crops"},
    {"state": "Kansas", "latitude": 38.5, "longitude": -98.0, "area": "Maximum Cultivated Crops"},
    {"state": "Illinois", "latitude": 40.0, "longitude": -89.0, "area": "Maximum Cultivated Crops"},
    {"state": "Indiana", "latitude": 40.0, "longitude": -86.0, "area": "Maximum Cultivated Crops"}
]


In [30]:
cities = pd.DataFrame(markers)

In [31]:
m.add_points_from_xy(
    cities,
    x="longitude",  # Column name for longitude
    y="latitude",   # Column name for latitude
    color_column="state",  # Use the "state" column to differentiate colors
    icon_names=["info", "map", "leaf", "globe", "star"],  # 5 icons for 5 states
    spin=True,  # Make icons spin (optional)
    add_legend=True,  # Add a legend
)


In [32]:
m.add_legend(title="NLCD Land Cover Type", builtin_legend="NLCD")

In [33]:
m

Map(center=[40, -95], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

In [34]:
m

Map(center=[40, -95], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

In [35]:
output_file = "map.html"
m.to_html(output_file)
print(f"Map saved as {output_file}")


Map saved as map.html
